## Imports and Dataframe Setup

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from scipy import stats
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso, ElasticNet


In [2]:
#CREATING DF FOR ALL CRIME

# Database credentials
username = 'crimeadmin'
password = '4DAnuuSBWVQ92w!F'
host = 'crimedbmysql.cspoouh9lugd.us-east-2.rds.amazonaws.com'
database = 'crimedb_mysql'
port = '3306'  # default MySQL port

# Create the database engine
engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}')

# SQL query or table name
query = "SELECT * FROM ML_ALLCRIME_EDU_INCOME"  #Name of table 

# Create a DataFrame from the SQL query
df_all = pd.read_sql(query, engine)



In [3]:
#CREATING DF FOR VIOLENT CRIME 

import pandas as pd
from sqlalchemy import create_engine

# Database credentials
username = 'crimeadmin'
password = '4DAnuuSBWVQ92w!F'
host = 'crimedbmysql.cspoouh9lugd.us-east-2.rds.amazonaws.com'
database = 'crimedb_mysql'
port = '3306'  # default MySQL port

# Create the database engine
engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}')

# SQL query or table name
query = "SELECT * FROM ML_VIOLENTCRIME_EDU_INCOME"  #Name of table 

# Create a DataFrame from the SQL query
df_violent = pd.read_sql(query, engine)


In [4]:
df_all.head()

,Sector,Month,Total Crimes,SAFETY,S_EDUCATION_LOW,S_EDUCATION_MEDIUM,S_EDUCATION_HIGH,S_INCOME_LOW,S_INCOME_MEDIUM,S_INCOME_HIGH,TRUST,T_EDUCATION_LOW,T_EDUCATION_MEDIUM,T_EDUCATION_HIGH,T_INCOME_LOW,T_INCOME_MEDIUM,T_INCOME_HIGH
0,110,2017-11,622,68.23,NaN,NaN,NaN,NaN,NaN,NaN,65.02,NaN,NaN,NaN,NaN,NaN,NaN
1,110,2017-12,620,65.67,NaN,NaN,NaN,NaN,NaN,NaN,64.76,NaN,NaN,NaN,NaN,NaN,NaN
2,110,2018-01,575,72.35,NaN,NaN,NaN,NaN,NaN,NaN,68.09,NaN,NaN,NaN,NaN,NaN,NaN
3,110,2018-02,524,75.66,NaN,NaN,NaN,NaN,NaN,NaN,70.62,NaN,NaN,NaN,NaN,NaN,NaN
4,110,2018-03,541,73.90,62.18,78.27,74.8,65.28,78.7,73.23,70.81,59.96,71.98,73.84,56.56,74.95,76.14


In [5]:
df_violent.head()

,Sector,Month,Violent Crimes,SAFETY,S_EDUCATION_LOW,S_EDUCATION_MEDIUM,S_EDUCATION_HIGH,S_INCOME_LOW,S_INCOME_MEDIUM,S_INCOME_HIGH,TRUST,T_EDUCATION_LOW,T_EDUCATION_MEDIUM,T_EDUCATION_HIGH,T_INCOME_LOW,T_INCOME_MEDIUM,T_INCOME_HIGH
0,110,2017-11,94,68.23,NaN,NaN,NaN,NaN,NaN,NaN,65.02,NaN,NaN,NaN,NaN,NaN,NaN
1,110,2017-12,72,65.67,NaN,NaN,NaN,NaN,NaN,NaN,64.76,NaN,NaN,NaN,NaN,NaN,NaN
2,110,2018-01,98,72.35,NaN,NaN,NaN,NaN,NaN,NaN,68.09,NaN,NaN,NaN,NaN,NaN,NaN
3,110,2018-02,95,75.66,NaN,NaN,NaN,NaN,NaN,NaN,70.62,NaN,NaN,NaN,NaN,NaN,NaN
4,110,2018-03,101,73.90,62.18,78.27,74.8,65.28,78.7,73.23,70.81,59.96,71.98,73.84,56.56,74.95,76.14


## Pearson Correlation Analysis

In [6]:
df_excluded = df_all.drop(columns=['Sector', 'Month'])

# Calculating Pearson correlation coefficients
correlation_matrix = df_excluded.corr()

# Extracting the correlation of each variable with 'Total Crimes'
correlation_with_total_crimes = correlation_matrix['Total Crimes'].drop('Total Crimes')

# Displaying the correlation coefficients
print(correlation_with_total_crimes)

SAFETY               -0.236950
S_EDUCATION_LOW      -0.164437
S_EDUCATION_MEDIUM   -0.204835
S_EDUCATION_HIGH     -0.132179
S_INCOME_LOW         -0.249899
S_INCOME_MEDIUM      -0.180409
S_INCOME_HIGH        -0.121271
TRUST                -0.297953
T_EDUCATION_LOW      -0.306289
T_EDUCATION_MEDIUM   -0.272278
T_EDUCATION_HIGH     -0.180600
T_INCOME_LOW         -0.280950
T_INCOME_MEDIUM      -0.263527
T_INCOME_HIGH        -0.161254
Name: Total Crimes, dtype: float64


In [7]:


# Assuming df_excluded is your dataframe with 'Total Crimes' and other columns
# First, let's calculate the Pearson correlation coefficients
correlation_matrix = df_excluded.corr()
correlation_with_total_crimes = correlation_matrix['Total Crimes'].drop('Total Crimes')

# Calculating the significance of each correlation coefficient
n = len(df_excluded)  # sample size
t_values = correlation_with_total_crimes.apply(lambda r: (r * np.sqrt(n - 2)) / np.sqrt(1 - r**2))
p_values = t_values.apply(lambda t: 2 * stats.t.sf(np.abs(t), n - 2))  # two-tailed p-values

# Combining correlation coefficients with their p-values
correlation_significance = pd.DataFrame({
    'Correlation': correlation_with_total_crimes,
    'p-value': p_values
})

# Displaying the results
correlation_significance



,Correlation,p-value
SAFETY,-0.236950,2.172223e-63
S_EDUCATION_LOW,-0.164437,5.457862e-31
S_EDUCATION_MEDIUM,-0.204835,1.735048e-47
S_EDUCATION_HIGH,-0.132179,1.656391e-20
S_INCOME_LOW,-0.249899,1.581867e-70
S_INCOME_MEDIUM,-0.180409,4.679319e-37
S_INCOME_HIGH,-0.121271,1.733433e-17
TRUST,-0.297953,7.689663e-101
T_EDUCATION_LOW,-0.306289,9.762979e-107
T_EDUCATION_MEDIUM,-0.272278,7.027985e-84


In [8]:
df_excluded = df_violent.drop(columns=['Sector', 'Month'])

# Calculating Pearson correlation coefficients
correlation_matrix = df_excluded.corr()

# Extracting the correlation of each variable with 'Total Crimes'
correlation_with_total_crimes = correlation_matrix['Violent Crimes'].drop('Violent Crimes')

# Displaying the correlation coefficients
print(correlation_with_total_crimes)

SAFETY               -0.490203
S_EDUCATION_LOW      -0.323947
S_EDUCATION_MEDIUM   -0.418849
S_EDUCATION_HIGH     -0.266230
S_INCOME_LOW         -0.410271
S_INCOME_MEDIUM      -0.395998
S_INCOME_HIGH        -0.235124
TRUST                -0.536334
T_EDUCATION_LOW      -0.450061
T_EDUCATION_MEDIUM   -0.474677
T_EDUCATION_HIGH     -0.342486
T_INCOME_LOW         -0.439241
T_INCOME_MEDIUM      -0.460519
T_INCOME_HIGH        -0.277398
Name: Violent Crimes, dtype: float64


In [9]:
# Assuming df_excluded is your dataframe with 'Total Crimes' and other columns
# First, let's calculate the Pearson correlation coefficients
correlation_matrix = df_excluded.corr()
correlation_with_total_crimes = correlation_matrix['Violent Crimes'].drop('Violent Crimes')

# Calculating the significance of each correlation coefficient
n = len(df_excluded)  # sample size
t_values = correlation_with_total_crimes.apply(lambda r: (r * np.sqrt(n - 2)) / np.sqrt(1 - r**2))
p_values = t_values.apply(lambda t: 2 * stats.t.sf(np.abs(t), n - 2))  # two-tailed p-values

# Combining correlation coefficients with their p-values
correlation_significance = pd.DataFrame({
    'Correlation': correlation_with_total_crimes,
    'p-value': p_values
})

# Displaying the results
correlation_significance

,Correlation,p-value
SAFETY,-0.490203,3.959288e-294
S_EDUCATION_LOW,-0.323947,7.026209e-120
S_EDUCATION_MEDIUM,-0.418849,4.437401e-207
S_EDUCATION_HIGH,-0.266230,3.841417e-80
S_INCOME_LOW,-0.410271,5.947132e-198
S_INCOME_MEDIUM,-0.395998,2.390863e-183
S_INCOME_HIGH,-0.235124,2.037301e-62
TRUST,-0.536334,0.000000e+00
T_EDUCATION_LOW,-0.450061,1.290717e-242
T_EDUCATION_MEDIUM,-0.474677,2.198090e-273


## Prepping for Machine Learing Model

In [10]:
# Converting the 'Month' column to datetime
df_violent['Month'] = pd.to_datetime(df_violent['Month'])


In [11]:
# Selecting numerical columns for standardization, excluding 'Sector' and 'Violent Crimes'
numerical_columns = df_violent.select_dtypes(include=['float64', 'int64']).columns
numerical_columns = numerical_columns.drop(['Sector', 'Violent Crimes'])

# Applying standardization
scaler = StandardScaler()
df_violent[numerical_columns] = scaler.fit_transform(df_violent[numerical_columns])

# Display the first few rows of the standardized dataset
df_violent.head()

,Sector,Month,Violent Crimes,SAFETY,S_EDUCATION_LOW,S_EDUCATION_MEDIUM,S_EDUCATION_HIGH,S_INCOME_LOW,S_INCOME_MEDIUM,S_INCOME_HIGH,TRUST,T_EDUCATION_LOW,T_EDUCATION_MEDIUM,T_EDUCATION_HIGH,T_INCOME_LOW,T_INCOME_MEDIUM,T_INCOME_HIGH
0,110,2017-11-01,94,1.279775,NaN,NaN,NaN,NaN,NaN,NaN,0.550908,NaN,NaN,NaN,NaN,NaN,NaN
1,110,2017-12-01,72,0.958365,NaN,NaN,NaN,NaN,NaN,NaN,0.523214,NaN,NaN,NaN,NaN,NaN,NaN
2,110,2018-01-01,98,1.797046,NaN,NaN,NaN,NaN,NaN,NaN,0.877906,NaN,NaN,NaN,NaN,NaN,NaN
3,110,2018-02-01,95,2.212620,NaN,NaN,NaN,NaN,NaN,NaN,1.147387,NaN,NaN,NaN,NaN,NaN,NaN
4,110,2018-03-01,101,1.991650,0.645498,2.230172,1.297663,0.891993,2.184311,1.098875,1.167625,0.052758,1.128835,0.840233,-0.027277,1.368428,0.88794


### Assessing stationarity of dataset
#### - Can explore more later but not super relevant 

In [12]:
# Extracting the 'Violent Crimes' series
violent_crimes_series = df_violent['Violent Crimes']

# Performing Augmented Dickey-Fuller test to check stationarity
adf_test = adfuller(violent_crimes_series.dropna())

# Displaying the results of the ADF test
adf_output = pd.Series(adf_test[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
for key, value in adf_test[4].items():
    adf_output[f'Critical Value ({key})'] = value

adf_output



Test Statistic                -6.096892e+00
p-value                        1.005048e-07
#Lags Used                     2.400000e+01
Number of Observations Used    4.866000e+03
Critical Value (1%)           -3.431695e+00
Critical Value (5%)           -2.862134e+00
Critical Value (10%)          -2.567086e+00
dtype: float64

## Trying different Time-Series Models

### ARIMA Model

In [13]:

# Splitting the dataset into train and test sets
train = violent_crimes_series.iloc[:-12]  # All data except the last 12 months
test = violent_crimes_series.iloc[-12:]   # Last 12 months

# Fitting the ARIMA model
model = ARIMA(train, order=(1, 0, 1))  # Starting with ARIMA(1,0,1)
fitted_model = model.fit()

# Summary of the model
fitted_model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:         Violent Crimes   No. Observations:                 4879
Model:                 ARIMA(1, 0, 1)   Log Likelihood              -21867.334
Date:                Wed, 17 Jan 2024   AIC                          43742.668
Time:                        13:27:33   BIC                          43768.639
Sample:                             0   HQIC                         43751.781
                               - 4879                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         93.9642      3.801     24.723      0.000      86.515     101.413
ar.L1          0.9457      0.006    166.139      0.000       0.935       0.957
ma.L1         -0.3965      0.013    -30.903      0.000      -0.422      -0.371
sigma2       457.4367      6.813     67.140      0.000     444.083     470.790
===================================================================================
Ljung-Box (L1) (Q):                   2.73   Jarque-Bera (JB):               880.36
Prob(Q):                              0.10   Prob(JB):                         0.00
Heteroskedasticity (H):               1.53   Skew:                            -0.14
Prob(H) (two-sided):                  0.00   Kurtosis:                         5.06
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Sarimax Model

In [14]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Fitting the SARIMA model with potential yearly seasonality
sarima_model = SARIMAX(violent_crimes_series, order=(1, 0, 1), seasonal_order=(0, 0, 0, 12))
sarima_fitted = sarima_model.fit(disp=False)

# Summary of the SARIMA model
sarima_fitted.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:         Violent Crimes   No. Observations:                 4891
Model:               SARIMAX(1, 0, 1)   Log Likelihood              -21978.625
Date:                Wed, 17 Jan 2024   AIC                          43963.250
Time:                        13:27:33   BIC                          43982.736
Sample:                             0   HQIC                         43970.087
                               - 4891                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9930      0.002    651.919      0.000       0.990       0.996
ma.L1         -0.4399      0.011    -39.652      0.000      -0.462      -0.418
sigma2       468.1607      6.760     69.250      0.000     454.910     481.411
===================================================================================
Ljung-Box (L1) (Q):                   6.51   Jarque-Bera (JB):              1134.21
Prob(Q):                              0.01   Prob(JB):                         0.00
Heteroskedasticity (H):               1.54   Skew:                            -0.33
Prob(H) (two-sided):                  0.00   Kurtosis:                         5.27
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Time-Series are scary. Let's try a time-series regression instead

## Regression Models excluding dates before March 2018 (missing demographic specific data)

### Creating a new dataframe. Removing months and adding lag features for additional time-series analysis for a regression

In [15]:
df_violent.set_index('Month', inplace=True)

# Filtering the data to include only dates from March 2018 onwards
df_violent_filtered = df_violent[df_violent.index >= '2018-03-01']

# Creating 1-month lagged features for numerical columns except 'Violent Crimes' and excluding 'Sector'
numerical_columns = df_violent_filtered.select_dtypes(include=['float64', 'int64']).columns
numerical_columns = [col for col in numerical_columns if col != 'Violent Crimes' and col != 'Sector']

for column in numerical_columns:
    df_violent_filtered[f'{column}_lag1'] = df_violent_filtered[column].shift(1)

# Dropping the initial row with NaN values due to lagging
df_violent_filtered = df_violent_filtered.dropna()

# Displaying the first few rows of the prepared dataset
df_violent_filtered.head()



/var/folders/gw/kyktsykx5kddft5zdz_8y75h0000gn/T/ipykernel_28963/1507532504.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_violent_filtered[f'{column}_lag1'] = df_violent_filtered[column].shift(1)
/var/folders/gw/kyktsykx5kddft5zdz_8y75h0000gn/T/ipykernel_28963/1507532504.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_violent_filtered[f'{column}_lag1'] = df_violent_filtered[column].shift(1)
/var/folders/gw/kyktsykx5kddft5zdz_8y75h0000gn/T/ipykernel_28963/1507532504.py:11: SettingWithCopyWa

,Sector,Violent Crimes,SAFETY,S_EDUCATION_LOW,S_EDUCATION_MEDIUM,S_EDUCATION_HIGH,S_INCOME_LOW,S_INCOME_MEDIUM,S_INCOME_HIGH,TRUST,...,S_INCOME_LOW_lag1,S_INCOME_MEDIUM_lag1,S_INCOME_HIGH_lag1,TRUST_lag1,T_EDUCATION_LOW_lag1,T_EDUCATION_MEDIUM_lag1,T_EDUCATION_HIGH_lag1,T_INCOME_LOW_lag1,T_INCOME_MEDIUM_lag1,T_INCOME_HIGH_lag1
Month,,,,,,,,,,,,,,,,,,,,,
2018-04-01,110,87,1.655173,1.144687,1.678584,1.159504,0.494996,2.127500,0.935931,0.897079,...,0.891993,2.184311,1.098875,1.167625,0.052758,1.128835,0.840233,-0.027277,1.368428,0.887940
2018-05-01,110,96,2.105901,1.419731,2.693111,0.813076,0.651208,2.851046,1.368130,0.511497,...,0.494996,2.127500,0.935931,0.897079,-0.171098,0.996959,0.642263,-1.026521,1.473232,0.771547
2018-06-01,110,89,2.120968,1.625279,2.317725,1.060524,1.116859,2.038530,1.656263,0.644640,...,0.651208,2.851046,1.368130,0.511497,-0.283915,0.714106,0.264570,-0.575989,1.089254,-0.010649
2018-07-01,110,109,1.763147,0.857898,1.726739,1.094549,0.890003,1.409313,1.460531,1.015310,...,1.116859,2.038530,1.656263,0.644640,0.079407,0.916923,-0.273689,0.495743,-0.209043,0.548626
2018-08-01,110,113,1.927619,1.209288,1.649035,1.675022,0.981541,1.282827,1.442647,1.385979,...,0.890003,1.409313,1.460531,1.015310,0.963282,1.006053,0.180638,1.180258,0.162286,0.900763


In [16]:

# Creating 1-month lagged features for numerical columns except 'Violent Crimes' and excluding 'Sector'
numerical_columns = df_violent_filtered.select_dtypes(include=['float64', 'int64']).columns
numerical_columns = [col for col in numerical_columns if col != 'Violent Crimes' and col != 'Sector']

for column in numerical_columns:
    df_violent_filtered[f'{column}_lag1'] = df_violent_filtered[column].shift(1)

# Dropping the initial row with NaN values due to lagging
df_violent_filtered = df_violent_filtered.dropna()

# Displaying the first few rows of the prepared dataset
df_violent_filtered.head()


,Sector,Violent Crimes,SAFETY,S_EDUCATION_LOW,S_EDUCATION_MEDIUM,S_EDUCATION_HIGH,S_INCOME_LOW,S_INCOME_MEDIUM,S_INCOME_HIGH,TRUST,...,S_INCOME_LOW_lag1_lag1,S_INCOME_MEDIUM_lag1_lag1,S_INCOME_HIGH_lag1_lag1,TRUST_lag1_lag1,T_EDUCATION_LOW_lag1_lag1,T_EDUCATION_MEDIUM_lag1_lag1,T_EDUCATION_HIGH_lag1_lag1,T_INCOME_LOW_lag1_lag1,T_INCOME_MEDIUM_lag1_lag1,T_INCOME_HIGH_lag1_lag1
Month,,,,,,,,,,,,,,,,,,,,,
2018-06-01,110,89,2.120968,1.625279,2.317725,1.060524,1.116859,2.038530,1.656263,0.644640,...,0.494996,2.127500,0.935931,0.897079,-0.171098,0.996959,0.642263,-1.026521,1.473232,0.771547
2018-07-01,110,109,1.763147,0.857898,1.726739,1.094549,0.890003,1.409313,1.460531,1.015310,...,0.651208,2.851046,1.368130,0.511497,-0.283915,0.714106,0.264570,-0.575989,1.089254,-0.010649
2018-08-01,110,113,1.927619,1.209288,1.649035,1.675022,0.981541,1.282827,1.442647,1.385979,...,1.116859,2.038530,1.656263,0.644640,0.079407,0.916923,-0.273689,0.495743,-0.209043,0.548626
2018-09-01,110,110,1.774447,0.525105,1.280216,1.502839,0.332814,1.056652,1.479409,1.193189,...,0.890003,1.409313,1.460531,1.015310,0.963282,1.006053,0.180638,1.180258,0.162286,0.900763
2018-10-01,110,107,1.518322,1.163284,1.261611,1.006910,0.181577,0.404924,1.469473,0.855538,...,0.981541,1.282827,1.442647,1.385979,0.499581,1.276173,0.832022,0.889385,0.146927,1.245008


In [17]:
df_violent_filtered.columns


Index(['Sector', 'Violent Crimes', 'SAFETY', 'S_EDUCATION_LOW',
       'S_EDUCATION_MEDIUM', 'S_EDUCATION_HIGH', 'S_INCOME_LOW',
       'S_INCOME_MEDIUM', 'S_INCOME_HIGH', 'TRUST', 'T_EDUCATION_LOW',
       'T_EDUCATION_MEDIUM', 'T_EDUCATION_HIGH', 'T_INCOME_LOW',
       'T_INCOME_MEDIUM', 'T_INCOME_HIGH', 'SAFETY_lag1',
       'S_EDUCATION_LOW_lag1', 'S_EDUCATION_MEDIUM_lag1',
       'S_EDUCATION_HIGH_lag1', 'S_INCOME_LOW_lag1', 'S_INCOME_MEDIUM_lag1',
       'S_INCOME_HIGH_lag1', 'TRUST_lag1', 'T_EDUCATION_LOW_lag1',
       'T_EDUCATION_MEDIUM_lag1', 'T_EDUCATION_HIGH_lag1', 'T_INCOME_LOW_lag1',
       'T_INCOME_MEDIUM_lag1', 'T_INCOME_HIGH_lag1', 'SAFETY_lag1_lag1',
       'S_EDUCATION_LOW_lag1_lag1', 'S_EDUCATION_MEDIUM_lag1_lag1',
       'S_EDUCATION_HIGH_lag1_lag1', 'S_INCOME_LOW_lag1_lag1',
       'S_INCOME_MEDIUM_lag1_lag1', 'S_INCOME_HIGH_lag1_lag1',
       'TRUST_lag1_lag1', 'T_EDUCATION_LOW_lag1_lag1',
       'T_EDUCATION_MEDIUM_lag1_lag1', 'T_EDUCATION_HIGH_lag1_lag1',
    

In [18]:
#Splitting into training and testing data

# Separating features and target variable
X = df_violent_filtered.drop(['Violent Crimes', 'Sector'], axis=1)
y = df_violent_filtered['Violent Crimes']

# Splitting the dataset into training and testing sets 
split_idx = int(len(df_violent_filtered) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]




In [19]:

# Initializing the Linear Regression model
lr_model = LinearRegression()

# Training the model on the training set
lr_model.fit(X_train, y_train)

# Making predictions on the test set
y_pred = lr_model.predict(X_test)

# Evaluating the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Displaying the evaluation metrics
mae, mse, rmse, r2


(26.348312667396904,
 1088.3105344610378,
 32.989551898457755,
 0.3190250873447268)

### Looking at other models

In [20]:
#Iterating through dataset to assess different models at once

# Initializing the models
rf_model = RandomForestRegressor()
gb_model = GradientBoostingRegressor()
svr_model = SVR()
ridge_model = Ridge()
lasso_model = Lasso()
elastic_net_model = ElasticNet()

# Dictionary to store models and their predictions
models = {
    "Random Forest": rf_model,
    "Gradient Boosting": gb_model,
    "Support Vector Regression": svr_model,
    "Ridge Regression": ridge_model,
    "Lasso Regression": lasso_model,
    "Elastic Net": elastic_net_model
}

# Dictionary to store evaluation metrics
model_metrics = {}

# Training each model and evaluating
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    model_metrics[model_name] = {
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "R2": r2
    }

# Displaying the evaluation metrics for each model
model_metrics



{'Random Forest': {'MAE': 25.674166666666668,
  'MSE': 1030.186239069264,
  'RMSE': 32.09651443800813,
  'R2': 0.35539447432044335},
 'Gradient Boosting': {'MAE': 25.96016053676957,
  'MSE': 1053.3519962510463,
  'RMSE': 32.45538470348251,
  'R2': 0.3408992553788479},
 'Support Vector Regression': {'MAE': 25.95524750241338,
  'MSE': 1083.945956832978,
  'RMSE': 32.92333453392864,
  'R2': 0.32175607980959076},
 'Ridge Regression': {'MAE': 26.347128213814024,
  'MSE': 1088.159106909976,
  'RMSE': 32.98725673513904,
  'R2': 0.319119838208651},
 'Lasso Regression': {'MAE': 26.56628163714372,
  'MSE': 1091.1623424888828,
  'RMSE': 33.03274651749204,
  'R2': 0.31724066124465855},
 'Elastic Net': {'MAE': 26.755320452183685,
  'MSE': 1101.561484969151,
  'RMSE': 33.18977982706651,
  'R2': 0.31073373613647026}}

### Random Forest the best performing model. Let's proceed with this model

In [21]:
# Initializing the Random Forest regressor
rf_model = RandomForestRegressor()

# Training the model on the training set
rf_model.fit(X_train, y_train)

# Making predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluating the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Displaying the evaluation metrics
mae, mse, rmse, r2


(25.5524025974026, 1026.6202692640693, 32.04091554971657, 0.35762576391999357)

## Analysis and Model Modifications
### (Needs further hyperparemeter tuning, feature extraction, cross-validation, etc.)

### Using just the features with strongest feature selection did not really improve R2 score. Need to find some other way to improve R2 score, use other dataset with all or aggregated crime types

## Crime Forecasts/Prediction